In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)
indicators = Indicators(alpaca_api)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]

def get_losers():
    print("Losers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(10), "Change".ljust(10), "Pct Gain".ljust(10), "MACD Buy?".ljust(10), "MFI".ljust(10), "Stochastic Oscillator")
    print("{:<30}".format("–" * 45))

    losers = alpaca_api.polygon.gainers_losers("losers")
    
    for symbol in range(len(losers)):
        
        ticker = losers[symbol].ticker
        bars = alpaca_api.get_barset(ticker, "1D", after=backdate)
        dataframe = selector.extract_bar_data(bars, ticker)
        _macd = indicators.get_macd(dataframe)
        macd = _macd["MACD"]
        signal = _macd["SIGNAL"]
        vzo = indicators.get_vzo(dataframe)
        stoch = indicators.get_stoch(dataframe)
        
        try:
            buysignal = macd.iloc[-1] < 0 and min(macd.iloc[-4:-2]) < signal.iloc[-1] and macd.iloc[-1] > signal.iloc[-1]
        except IndexError:
            # Throwing away due to index errors, will handle later
            continue
        
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),            
            "${:.2f}".format(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade["p"]).ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
            "{:.2f}%".format(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc).ljust(10),  
        )

def get_gainers():
    print("Gainers".center(45))
    print()
    print("Ticker".ljust(10), "Last".ljust(10), "Change".ljust(10), "% Change".ljust(10), "MACD Buy?".ljust(10), "MFI".ljust(10), "VZO".ljust(10), "Stochastic Oscillator")
    print("{:<30}".format("–" * 45))
    
    gainers = alpaca_api.polygon.gainers_losers()
    
    for symbol in range(len(gainers)):
        
        ticker = gainers[symbol].ticker
        bars = alpaca_api.get_barset(ticker, "1D", after=backdate)
        dataframe = selector.extract_bar_data(bars, ticker)
        _macd = indicators.get_macd(dataframe)
        macd = _macd["MACD"]
        signal = _macd["SIGNAL"]
        mfi = indicators.get_mfi(dataframe)
        vzo = indicators.get_vzo(dataframe)
        stoch = indicators.get_stoch(dataframe)
        
        try:
            buysignal = macd.iloc[-1] < 0 and min(macd.iloc[-4:-2]) < signal.iloc[-1] and macd.iloc[-1] > signal.iloc[-1]
        except IndexError:
            # Throwing away due to index errors, will handle later
            continue
        
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers()[symbol].lastTrade["p"]).ljust(10),
            "${:.2f}".format(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
            "{:.2f}%".format(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc).ljust(10), 
            str(buysignal).ljust(10),
            "{:.2f}".format(mfi.iloc[-1]).ljust(10),
            "{:.2f}".format(vzo.iloc[-1]).ljust(10),
            "{:.2f}".format(stoch.iloc[-1]).ljust(10)
        )

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\benjamin.beale\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
if __name__ == "__main__":
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Ticker     Last       Change     % Change   MACD Buy?  MFI        VZO        Stochastic Oscillator
–––––––––––––––––––––––––––––––––––––––––––––
AMHpF      $25.45     $25.27     14459.50%  False      27.38      -41.27     10.85     
MHpC       $5.93      $5.78      3853.33%   False      86.64      55.68      83.90     
DRIO       $4.85      $4.64      2260.10%   False      60.96      -8.10      90.64     
BNTC       $5.70      $5.07      804.76%    False      67.57      13.78      80.62     
KRTX       $88.00     $70.32     397.74%    False      99.41      95.51      97.33     
TYp        $53.72     $42.16     364.70%    True       24.94      -27.97     46.20     


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
ticker = "FTCH"

In [ ]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

In [ ]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

In [ ]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format("–" * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

In [ ]:
# alpaca_api.polygon.historic_trades(ticker, "2019-10-10", limit=10)
# alpaca_api.polygon.news(ticker)
# alpaca_api.polygon.last_trade(ticker)
# alpaca_api.polygon.last_quote(ticker)
# alpaca_api.polygon.company(ticker)
# alpaca_api.polygon.dividends(ticker)
# alpaca_api.polygon.financials(ticker)

In [ ]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [ ]:
stocktwits

In [ ]:
tweets 

In [ ]:
tenquant_data